In [8]:
import requests
from config import DATA_DOWNLOAD_URL
import pandas as pd
import io
import gzip
import numpy as np
from datetime import datetime

In [9]:
def get_market_year(row): #make a market year row in datas
    year = row['DATE'].year
    month = row['DATE'].month
    if month >= 9:
        market_year = f"{year}/{year + 1}"
    else:
        market_year = f"{year - 1}/{year}"
    return market_year

In [7]:
df = pd.DataFrame(columns=["DATE", "DEP", "RR", "TN", "TX", "TM"])#, "UM"])

In [ ]:
for i in range(95):
    print(i)
    url = f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_{i+1:02d}_previous-1950-2023_RR-T-Vent.csv.gz"
    response = requests.get(url)

    compressed_data = io.BytesIO(response.content)
    decompressed_data = gzip.GzipFile(fileobj=compressed_data)

    tmp = pd.read_csv(decompressed_data, sep=";")
    tmp["DATE"] = pd.to_datetime(tmp["AAAAMMJJ"], format="%Y%m%d") #string to date
    tmp['DEP'] = i+1
    tmp = tmp[["DATE", "DEP", "RR", "TN", "TX", "TM"]] #keep only, in order : Date, departement code, précipitations, temp min, temp max and moy temp
    df = pd.concat([df, tmp])
df.to_csv(f"{DATA_DOWNLOAD_URL}/raw_rr_tn_tx_tm-1950-2023.csv")

In [ ]:
#data for current market year
for i in range(95):
    print(i)
    url = f"https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_{i+1:02d}_latest-2024-2025_RR-T-Vent.csv.gz"
    response = requests.get(url)

    compressed_data = io.BytesIO(response.content)
    decompressed_data = gzip.GzipFile(fileobj=compressed_data)

    tmp = pd.read_csv(decompressed_data, sep=";")
    tmp["DATE"] = pd.to_datetime(tmp["AAAAMMJJ"], format="%Y%m%d") #string to date
    tmp['DEP'] = i+1
    tmp = tmp[["DATE", "DEP", "RR", "TN", "TX", "TM"]] #keep only, in order : Date, departement code, précipitations, temp min, temp max and moy temp
    df = pd.concat([df, tmp])
df.to_csv(f"{DATA_DOWNLOAD_URL}/raw_rr_tn_tx_tm-current.csv")

Make a mean file for x last year and fo each month, each departement

In [10]:
df = pd.read_csv(f"{DATA_DOWNLOAD_URL}/raw_rr_tn_tx_tm-1950-2023.csv")
df["DATE"] = pd.to_datetime(df["DATE"])
df = df[df["DATE"] >= "2019-09-01"] #year to start, i will take arbitrary last 5 market years
df = df.drop(columns=["Unnamed: 0"])
df["MY"] = df.apply(get_market_year, axis=1)
df["MONTH"] = df["DATE"].dt.month
df["YEAR-MONTH"] = df["DATE"].dt.to_period("M")

current = df[df["MY"] == "2024/2025"].copy() #copy current MY
df = df[df["MY"] != "2024/2025"] #remove current MY

In [13]:
temp = df[["TN", "TX", "TM", "DEP", "MONTH"]].groupby(["DEP", "MONTH"]).mean().reset_index()
meanDailyPrecip = df[["RR", "DEP", "DATE"]].groupby(["DEP", "DATE"]).mean().reset_index()
meanDailyPrecip["YEAR-MONTH"] = pd.to_datetime(meanDailyPrecip["DATE"]).dt.to_period("M")
sumMonthlyPrecip = meanDailyPrecip[["DEP", "RR", "YEAR-MONTH"]].groupby(["DEP", "YEAR-MONTH"]).sum().reset_index()
sumMonthlyPrecip["MONTH"] = sumMonthlyPrecip["YEAR-MONTH"].dt.month
precip = sumMonthlyPrecip[["MONTH", "RR", "DEP"]].groupby(["DEP", "MONTH"]).mean().reset_index()

In [14]:
weather = pd.merge(temp, precip, on=["DEP", "MONTH"], how="inner")

In [7]:
weather.to_csv("../YieldModel/data/forecast/currentYear/historic_mean_weather.csv")

Same for the current year

In [16]:
temp = current[["TN", "TX", "TM", "DEP", "MONTH"]].groupby(["DEP", "MONTH"]).mean().reset_index()
meanDailyPrecip = current[["RR", "DEP", "DATE"]].groupby(["DEP", "DATE"]).mean().reset_index()
meanDailyPrecip["YEAR-MONTH"] = pd.to_datetime(meanDailyPrecip["DATE"]).dt.to_period("M")
sumMonthlyPrecip = meanDailyPrecip[["DEP", "RR", "YEAR-MONTH"]].groupby(["DEP", "YEAR-MONTH"]).sum().reset_index()
sumMonthlyPrecip["MONTH"] = sumMonthlyPrecip["YEAR-MONTH"].dt.month
precip = sumMonthlyPrecip[["MONTH", "RR", "DEP"]].groupby(["DEP", "MONTH"]).mean().reset_index()

In [17]:
currentWeather = pd.merge(temp, precip, on=["DEP", "MONTH"], how="inner")

In [19]:
currentWeather.to_csv("../YieldModel/data/forecast/currentYear/current_mean_weather.csv")